In [1]:
from IPython.display import display

from io import StringIO
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
bid_df = pd.read_csv('20201007_bids_cb.csv')
offer_df = pd.read_csv('20201007_da_co.csv')

In [3]:
ts = '10/07/2020 11:00:00'
bid_df = bid_df.loc[bid_df['Date/Time Beginning (EST)'] == ts]
offer_df = offer_df.loc[offer_df['Date/Time Beginning (EST)'] == ts]


In [4]:
bid_df = bid_df.loc[~bid_df.PRICE1.isnull()]
bid_df.groupby('Type of Bid').head(1)

,Region,Market Participant Code,Date/Time Beginning (EST),Date/Time End (EST),MW,LMP,Type of Bid,Bid ID,PRICE1,MW1,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9
1541,NaN,122062269,10/07/2020 11:00:00,10/07/2020 12:00:00,5.0,24.11,I,3959349603,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1542,NaN,122062269,10/07/2020 11:00:00,10/07/2020 12:00:00,0.0,25.18,D,3742678325,20.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17387,Central,122062313,10/07/2020 11:00:00,10/07/2020 12:00:00,225.1,24.23,P,450524077,185.0,225.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def parse_bid_curve(row):
    bids = (
        pd.DataFrame({
            'price': [row['PRICE%d' % i] for i in range(1, 10)],
            'volume': [row['MW%d' % i] for i in range(1, 10)]},
            index=range(1, 10)
        )
        .dropna(how='any', axis='rows')
        .sort_values('price')
        .drop_duplicates('price', keep='last')
    )
    return bids

def validate_bid(row):
    bids = parse_bid_curve(row)
    if len(bids) == 0:
        return False
    return bids.price.is_monotonic_increasing and bids.volume.is_monotonic_decreasing

In [6]:
valid_bid_df = bid_df.loc[bid_df.apply(validate_bid, axis='columns')]

In [7]:
valid_bid_df.groupby('Date/Time Beginning (EST)').count()

,Region,Market Participant Code,Date/Time End (EST),MW,LMP,Type of Bid,Bid ID,PRICE1,MW1,PRICE2,...,PRICE5,MW5,PRICE6,MW6,PRICE7,MW7,PRICE8,MW8,PRICE9,MW9
Date/Time Beginning (EST),,,,,,,,,,,,,,,,,,,,,
10/07/2020 11:00:00,5989,8058,8058,8058,8058,8058,8058,8058,8058,1889,...,145,145,64,64,44,44,26,26,21,21


In [8]:
def parse_offer_curve(row):
    offers = (
        pd.DataFrame({
            'price': [row['Price%d' % i] for i in range(1, 10)],
            'volume': [row['MW%d' % i] for i in range(1, 10)]},
            index=range(1, 10)
        )
        .dropna(how='any', axis='rows')
        .sort_values('price')
        .drop_duplicates('price', keep='last')
    )
    return offers

def validate_offer(row):
    offers = parse_offer_curve(row)
    if len(offers) == 0:
        return False
    return offers.price.is_monotonic_decreasing and offers.volume.is_monotonic_increasing

In [9]:
valid_offer_df = offer_df.loc[offer_df.apply(validate_offer, axis='columns')]

In [10]:
valid_offer_df.groupby('Date/Time Beginning (EST)').count()

,Region,Unit Code,Date/Time End (EST),Economic Max,Economic Min,Emergency Max,Emergency Min,Economic Flag,Emergency Flag,Must Run Flag,...,MW6,Price7,MW7,Price8,MW8,Price9,MW9,Price10,MW10,Slope
Date/Time Beginning (EST),,,,,,,,,,,,,,,,,,,,,
10/07/2020 11:00:00,397,397,397,397,397,397,397,397,397,397,...,29,24,24,24,24,23,23,23,23,397


In [11]:
ts = valid_offer_df.query('`Date/Time Beginning (EST)` == "10/07/2020 11:00:00"')
ts.loc[~ts['MW10'].isnull()]

,Region,Unit Code,Date/Time Beginning (EST),Date/Time End (EST),Economic Max,Economic Min,Emergency Max,Emergency Min,Economic Flag,Emergency Flag,...,MW6,Price7,MW7,Price8,MW8,Price9,MW9,Price10,MW10,Slope
14100,North,A4021,10/07/2020 11:00:00,10/07/2020 12:00:00,16.0,8.0,25.6,6.0,0,0,...,12.5,26.23,13.4,26.23,14.2,26.23,15.1,26.23,16.0,1
14151,Central,A4181,10/07/2020 11:00:00,10/07/2020 12:00:00,20.0,10.0,20.0,10.0,0,0,...,16.1,90.39,17.3,90.39,18.6,90.39,19.8,90.39,21.0,1
14157,Central,A4196,10/07/2020 11:00:00,10/07/2020 12:00:00,0.0,0.0,0.0,0.0,0,0,...,0.6,16.23,0.7,16.23,0.8,16.23,0.9,16.23,1.0,1
14160,Central,A4202,10/07/2020 11:00:00,10/07/2020 12:00:00,0.0,0.0,0.0,0.0,0,0,...,0.6,33.80,0.7,33.80,0.8,33.80,0.9,33.80,1.0,1
14163,Central,A4205,10/07/2020 11:00:00,10/07/2020 12:00:00,0.0,0.0,0.0,0.0,0,0,...,0.6,33.80,0.7,33.80,0.8,33.80,0.9,33.80,1.0,1
14170,Central,A4240,10/07/2020 11:00:00,10/07/2020 12:00:00,1.0,0.0,1.0,0.0,0,0,...,1.1,-34.36,1.3,-34.36,1.6,-34.36,1.8,-34.36,2.0,1
14172,Central,A4260,10/07/2020 11:00:00,10/07/2020 12:00:00,0.0,0.0,0.0,0.0,0,0,...,0.6,19.04,0.7,19.04,0.8,19.04,0.9,19.04,1.0,1
14371,Central,A5616,10/07/2020 11:00:00,10/07/2020 12:00:00,61.0,0.0,61.0,0.0,0,0,...,34.4,0.00,41.3,0.00,48.2,0.00,55.1,0.00,62.0,1
14446,North,A5970,10/07/2020 11:00:00,10/07/2020 12:00:00,4.4,1.0,29.4,1.0,1,0,...,25.0,0.00,30.0,0.00,35.0,0.00,40.0,0.00,50.0,0
14459,Central,A6042,10/07/2020 11:00:00,10/07/2020 12:00:00,47.0,0.0,96.0,0.0,1,0,...,60.0,0.00,70.0,0.00,80.0,0.00,90.0,0.00,100.0,1


In [14]:
unpacked_offers = []
for offer_idx, offer in valid_offer_df.iterrows():
    
unpacked_offers = pd.DataFrame(unpacked_offers, columns=['id', 'node', 'capacity (MW)', 'offer ($/MW)'])